In [2]:
import numpy as np
import json
import requests
import time
#import urllib PYTHON 2 Sorry :D
import urlparse
import config
from nlp import *
import os.path
#from preproc import preproc --> Because of Memory
import pandas as pd
# for Word2Vec model Load
from gensim.models import KeyedVectors

URL = "https://api.telegram.org/bot{}/".format(config.TOKEN)

In [3]:
#///////////////////////////////////////////////
def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates(offset=None):
    url = URL + "getUpdates?timeout=100"
    print("updates")
    if offset:
    	url += "&offset={}".format(offset)
    js = get_json_from_url(url)
    return js

def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)

def send_message(text, chat_id, reply_markup=None):
    #text = urllib.parse.quote_plus(text) for Python 2
    #text = urlparse.urlparse(text)
    url = URL + "sendMessage?text={}&chat_id={}&parse_mode=Markdown".format(text, chat_id)
    if reply_markup:
        url += "&reply_markup={}".format(reply_markup)
    get_url(url)

def build_keyboard(items):
    keyboard = [[item] for item in items]
    reply_markup = {"keyboard":keyboard, "one_time_keyboard": True}
    return json.dumps(reply_markup)
    
def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)

def handle_updates(updates, nlp):
    for update in updates["result"]:
        try:
            text = update["message"]["text"]
            chat = update["message"]["chat"]["id"]
            #items = db.get_items(chat)
            message = nlp.respond(text)
            print("response = " + message)
            send_message(message, chat)
        except KeyError:
            pass
        
def load_zipped_pickle(filename):	# loads and unpacks
    with gzip.open(filename, 'rb') as f:
        loaded_object = cPickle.load(f)
        f.close()
        return loaded_object

def main():
    last_update_id = None
    #if not os.path.isfile("proc_data/proc_data.csv"): # Outcommented because of memory
    #    preproc()
    df = pd.DataFrame.from_csv("proc_data/proc_data.csv")
    nlp = NLP(df)
    print("The bot is now active")
    while True:
        updates = get_updates(last_update_id)
        if len(updates["result"]) > 0:
            last_update_id = get_last_update_id(updates) + 1
            handle_updates(updates, nlp)
        time.sleep(0.5)

In [5]:
# Split the Main Function to not always load again
# Stops at 6.6 GB for me :D

EMBEDDING_FILE = './model/GoogleNews-vectors-negative300.bin'
w2vmodel = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
description_vecs = load_zipped_pickle('./descriptions_vec.npy')
#save_zipped_pickle(descriptions_vec,'descriptions_vec.npy')
last_update_id = None
#if not os.path.isfile("proc_data/proc_data.csv"): # Outcommented because of memory
#    preproc()
df = pd.DataFrame.from_csv("proc_data/proc_data.csv")

INFO:gensim.models.keyedvectors:loading projection weights from ./model/GoogleNews-vectors-negative300.bin
INFO:gensim.models.keyedvectors:loaded (3000000, 300) matrix from ./model/GoogleNews-vectors-negative300.bin


In [1]:
#RELOAD CHANGES IN NLP FILE --> For Debugging
import nlp
reload(nlp)
from nlp import *

In [4]:
nlp = NLP(df, w2vmodel, description_vecs)
print "Loaded Word2Vec model + Descriptions_vec"
print("The bot is now active")

while True:
        updates = get_updates(last_update_id)
        if len(updates["result"]) > 0:
            last_update_id = get_last_update_id(updates) + 1
            handle_updates(updates, nlp)
        time.sleep(0.5)

NameError: name 'df' is not defined